In [46]:
from google.colab import drive
import json

# Monta Google Drive en el entorno de Colab
drive.mount('/content/drive')

# Ruta al archivo de contraseñas JSON en drive
file_path = '/content/drive/MyDrive/pass.json'

# Lee el archivo JSON
with open(file_path, 'r') as file:
    config_data = json.load(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
# Leer los datos de la API
import requests
import pandas as pd
from datetime import datetime, timedelta

# Acceder a las contraseñas de la api
api_key = config_data['api_key']

# Lista de nombres de países
paises = [
    'Afghanistan', 'Albania', 'Algeria', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Aruba', 'Austria',
    'Barbados', 'Belgium', 'Botswana', 'Brazil', 'Bulgaria', 'Canada', 'Colombia',
    'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Federated States of Micronesia', 'Finland',
    'France', 'Germany', 'Greece', 'Guyana', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel',
    'Italy', 'Jamaica', 'Kenya', 'Kuwait', 'Latvia', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malta',
    'Marshall Islands', 'Mexico', 'Moldova', 'Mongolia', 'Montenegro', 'Myanmar', 'Netherlands', 'New Zealand',
    'North Macedonia', 'Norway', 'Oman', 'Palau', 'Papua New Guinea', 'Paraguay', 'Philippines', 'Poland',
    'Portugal', 'Romania', 'Russia', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'Senegal', 'Serbia',
    'Slovakia', 'Slovenia', 'Solomon Islands', 'Spain', 'Suriname', 'Sweden', 'Switzerland',
    'Tanzania', 'Thailand', 'Tonga', 'Trinidad and Tobago', 'United Kingdom of Great Britain and Northern Ireland',
    'United States', 'Vanuatu', 'Zimbabwe'
]

# Lista para almacenar los datos de cada país
all_data = []
# Lista para almacenar los nombres de los países
countries = []
id = []

# Iterar sobre la lista de países
for i,pais in enumerate(paises):
    # Hacer la solicitud a la API para cada país
    url = f'http://api.weatherapi.com/v1/current.json?key={api_key}&q={pais}&aqi=no'
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta a JSON y agregar los datos al listado
        data_json = json.loads(response.text)
        all_data.append({
            'location': data_json['location'],
            'current': data_json['current'],
            'id': i + 1,
        })
    else:
        print(f"No se pudo obtener datos para {pais}. Código de estado: {response.status_code}")

In [48]:
# Convertir la lista de datos a un DataFrame
df = pd.json_normalize(all_data, sep='_')

# Mostrar el DataFrame
df

,id,location_name,location_region,location_country,location_lat,location_lon,location_tz_id,location_localtime_epoch,location_localtime,current_last_updated_epoch,...,current_precip_in,current_humidity,current_cloud,current_feelslike_c,current_feelslike_f,current_vis_km,current_vis_miles,current_uv,current_gust_mph,current_gust_kph
0,1,Kabul,Kabol,Afghanistan,34.52,69.18,Asia/Kabul,1709080794,2024-02-28 5:09,1709080200,...,0.00,91,76,-3.3,26.0,10.0,6.0,1.0,5.4,8.7
1,2,Tirana,Tirane,Albania,41.33,19.82,Europe/Tirane,1709080794,2024-02-28 1:39,1709080200,...,0.00,87,0,10.8,51.4,10.0,6.0,1.0,7.0,11.3
2,3,Algiers,Alger,Algeria,36.76,3.05,Africa/Algiers,1709080794,2024-02-28 1:39,1709080200,...,0.05,81,75,7.0,44.6,10.0,6.0,1.0,13.6,21.8
3,4,The Valley,,Anguilla,18.22,-63.05,America/Anguilla,1709080794,2024-02-27 20:39,1709080200,...,0.00,79,75,27.8,82.0,10.0,6.0,1.0,21.2,34.1
4,5,Saint John,Saint John,Antigua and Barbuda,17.12,-61.85,America/Antigua,1709080794,2024-02-27 20:39,1709080200,...,0.00,74,50,26.7,80.1,10.0,6.0,1.0,21.5,34.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,Port Of Spain,Port-of-Spain,Trinidad and Tobago,10.65,-61.52,America/Port_of_Spain,1709080798,2024-02-27 20:39,1709080200,...,0.00,70,25,29.5,85.1,10.0,6.0,1.0,18.6,30.0
77,78,Great Eppleton,Tyne and Wear,United Kingdom,54.83,-1.43,Europe/London,1709080872,2024-02-28 0:41,1709080200,...,0.00,87,0,0.6,33.0,10.0,6.0,1.0,15.9,25.7
78,79,United,Louisiana,United States of America,29.78,-91.75,America/Chicago,1709080798,2024-02-27 18:39,1709080200,...,0.00,93,0,20.0,68.0,10.0,6.0,6.0,20.5,32.9
79,80,Port Vila,Efate,Vanuatu,-17.73,168.32,Pacific/Efate,1709080811,2024-02-28 11:40,1709080200,...,0.00,75,25,37.9,100.2,10.0,6.0,9.0,11.9,19.1


In [49]:
# Ver la informacion del dataframe sin manipulacion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          81 non-null     int64  
 1   location_name               81 non-null     object 
 2   location_region             81 non-null     object 
 3   location_country            81 non-null     object 
 4   location_lat                81 non-null     float64
 5   location_lon                81 non-null     float64
 6   location_tz_id              81 non-null     object 
 7   location_localtime_epoch    81 non-null     int64  
 8   location_localtime          81 non-null     object 
 9   current_last_updated_epoch  81 non-null     int64  
 10  current_last_updated        81 non-null     object 
 11  current_temp_c              81 non-null     float64
 12  current_temp_f              81 non-null     float64
 13  current_is_day              81 non-nu

In [50]:
# Filtrado de datos inecesarios y undades anglosajones
df = df.drop(['location_tz_id',
              'location_localtime_epoch',
              'current_last_updated_epoch',
              'current_wind_mph',
              'current_condition_icon',
              'current_temp_f',
              'current_pressure_in',
              'current_precip_in',
              'current_gust_mph',
              'current_feelslike_f',
              'current_vis_miles'], axis=1)

In [51]:
# Ver la informacion del dataframe manipulados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      81 non-null     int64  
 1   location_name           81 non-null     object 
 2   location_region         81 non-null     object 
 3   location_country        81 non-null     object 
 4   location_lat            81 non-null     float64
 5   location_lon            81 non-null     float64
 6   location_localtime      81 non-null     object 
 7   current_last_updated    81 non-null     object 
 8   current_temp_c          81 non-null     float64
 9   current_is_day          81 non-null     int64  
 10  current_condition_text  81 non-null     object 
 11  current_condition_code  81 non-null     int64  
 12  current_wind_kph        81 non-null     float64
 13  current_wind_degree     81 non-null     int64  
 14  current_wind_dir        81 non-null     obje

In [52]:
# Conectarse a redshift
import psycopg2
from sqlalchemy import create_engine

# Conecta con Redshift
redshift_credentials = config_data['redshift_credentials'] # busca del archivo pass.json montado
conn = psycopg2.connect(
    dbname=redshift_credentials['dbname'],
    user=redshift_credentials['user'],
    password=redshift_credentials['password'],
    host=redshift_credentials['host'],
    port=redshift_credentials['port']
)

# Crea un motor para SQLAlchemy
engine = create_engine(f"postgresql://{redshift_credentials['user']}:{redshift_credentials['password']}@{redshift_credentials['host']}:{redshift_credentials['port']}/{redshift_credentials['dbname']}")

In [53]:
# Define la estructura de la tabla en redshift, dropear si existe y luego crearla
tabla_sql = '''
DROP TABLE IF EXISTS weather_data;
CREATE TABLE weather_data (
    id INT,
    location_name VARCHAR(255) PRIMARY KEY,
    location_region VARCHAR(255),
    location_country VARCHAR(255),
    location_lat FLOAT,
    location_lon FLOAT,
    location_localtime TIMESTAMP,
    current_last_updated TIMESTAMP,
    current_temp_c FLOAT,
    current_is_day BOOL,
    current_condition_text VARCHAR(255),
    current_condition_code INT,
    current_wind_kph FLOAT,
    current_wind_degree INT,
    current_wind_dir VARCHAR(255),
    current_pressure_mb FLOAT,
    current_precip_mm FLOAT,
    current_humidity INT,
    current_cloud INT,
    current_feelslike_c FLOAT,
    current_vis_km FLOAT,
    current_uv FLOAT,
    current_gust_kph FLOAT
) DISTSTYLE KEY DISTKEY(location_name) SORTKEY(location_name);
'''
try:
    # Ejecuta la consulta SQL para crear la tabla
    cursor = conn.cursor()
    cursor.execute("ROLLBACK")  # Realiza un rollback explícito para limpiar el estado de la transacción abortada
    conn.commit()  # Confirma el rollback
    cursor.execute(tabla_sql)  # Intenta ejecutar la consulta nuevamente
    conn.commit()  # Confirma la creación de la tabla
    print("Tabla creada exitosamente.")
except Exception as e:
    print("Error al crear la tabla:", e)
    conn.rollback()  # Realiza un rollback en caso de error

Tabla creada exitosamente.


In [54]:
# Insertar los datos en las tablas ya creadas
# Crea un cursor
cursor = conn.cursor()

# Consulta SQL para la inserción con todas las columnas
sql_insert = '''
INSERT INTO weather_data (
    id, location_name, location_region, location_country, location_lat, location_lon, location_localtime, current_last_updated, current_temp_c,
    current_is_day, current_condition_text, current_condition_code, current_wind_kph, current_wind_degree, current_wind_dir, current_pressure_mb,
    current_precip_mm, current_humidity, current_cloud, current_feelslike_c, current_vis_km, current_uv, current_gust_kph
) VALUES
'''

# Lista para contener los valores de todas las filas
values = []

for index, row in df.iterrows():
    # Extrae los valores de cada fila y los agrega a la lista
    values.append((
      row['id'], row['location_name'], row['location_region'], row['location_country'], row['location_lat'],
      row['location_lon'], row['location_localtime'], row['current_last_updated'], row['current_temp_c'],
      row['current_is_day'], row['current_condition_text'], row['current_condition_code'], row['current_wind_kph'],
      row['current_wind_degree'], row['current_wind_dir'], row['current_pressure_mb'], row['current_precip_mm'],
      row['current_humidity'], row['current_cloud'], row['current_feelslike_c'], row['current_vis_km'],
      row['current_uv'], row['current_gust_kph']
    ))

# Crea una lista de placeholders para los valores
placeholders = ', '.join(['%s'] * len(values[0]))

# Concatena los valores de todas las filas en una lista plana
flat_values = [item for sublist in values for item in sublist]

# Ejecuta la consulta SQL con todos los valores
cursor.execute(sql_insert + ', '.join(['(' + placeholders + ')'] * len(values)), flat_values)

# Confirma la transacción
conn.commit()

# Cierra el cursor y la conexión
cursor.close()
conn.close()
